In [100]:
import json
from collections import Counter

docs_metadata = json.load(open('../data/DocumentCloud/documents.json'))
TEXT_DATABASE_PATH = "../data/DocumentCloud/text/"

In [88]:
def get_docs_for_source(source, only_title=False):
    source_filter = filter((lambda x: x['source'].lower() == source.lower()), docs_metadata)
    if not only_title:
        source_docs = [x for x in source_filter]
    else:
        source_docs = [x['title'] for x in source_filter]
    print("Total documents for " +  str(source) + ": " + str(len(source_docs)))
    return source_docs

In [89]:
gaming_docs = get_docs_for_source("Illinois Gaming Board")

Total documents for Illinois Gaming Board: 683


In [91]:
def search_source_text(phrase, source, title_include=""):
    for doc in get_docs_for_source("Illinois Gaming Board"):
        search_doc_text(phrase, doc)

def search_doc_text(phrase, doc):
    fname = TEXT_DATABASE_PATH + doc['id'] + ".txt"
    with open(fname, "r") as f:
        searchlines = f.readlines()
    for i, line in enumerate(searchlines):
        if phrase in line:
            print("\n-------------------")
            print("Title: %s, Line: %d \n %s \n" % (doc['title'], i, doc['canonical_url']))
            for l in searchlines[i:i+3]:
                print(l)


In [92]:
search_source_text("misconduct", "Illinois Gaming Board")

Total documents for Illinois Gaming Board: 683

-------------------
Title: Riverboat/Video Gaming 2000-09-15 - Minutes: Riverboat, Line: 63 
 https://www.documentcloud.org/documents/6138716-Riverboat-Video-Gaming-2000-09-15-Minutes.html 

members and Administrator Acosta. Overall, the statement outlined alleged misconduct by

Argosy.

Anita R. Bedell, representing the Illinois Church Action on Alcohol Problems, was present to


-------------------
Title: Riverboat/Video Gaming 2000-09-15 - Minutes: Riverboat, Line: 287 
 https://www.documentcloud.org/documents/6138716-Riverboat-Video-Gaming-2000-09-15-Minutes.html 

misconduct pursuant to the Board Rule.

EMPRESS CASINO JOLIET – JOHN KEENAN, SURVEILLANCE DIRECTOR. –

LEVEL ONE


-------------------
Title: Riverboat/Video Gaming 2003-11-18 - Minutes: Riverboat, Line: 372 
 https://www.documentcloud.org/documents/6138171-Riverboat-Video-Gaming-2003-11-18-Minutes.html 

failed to notify his supervisor or report the misconduct referenced i

In [109]:
# Section extraction
def get_caps_lines(doc):
    fname = TEXT_DATABASE_PATH + doc['id'] + ".txt"
    with open(fname, "r") as f:
        searchlines = f.readlines()
    headers = []
    for i, line in enumerate(searchlines):
        if line.isupper():
            headers.append(line.strip('\n').strip())
    return headers

def get_source_caps_lines(source):
    source_docs = get_docs_for_source(source)
    allheaders = []
    for doc in source_docs:
        allheaders += get_caps_lines(doc)
    c = Counter(allheaders)
    sorted(c)
    return c.most_common()

In [113]:
get_caps_lines(gaming_docs[1])
get_source_caps_lines("Chicago Teachers Pension Fund")

Total documents for Chicago Teachers Pension Fund: 33


[('N/A', 778),
 ('M', 354),
 ('EDUC GENERAL', 118),
 ('OF THE', 96),
 ('JUNE 20', 91),
 ('OCTOBER 17', 72),
 ('GLOBAL EQUITY', 60),
 ('SPEC SVCS SUP', 55),
 ('DOMESTIC EQUITY', 53),
 ('FIXED INCOME', 50),
 ('PROCEEDINGS', 48),
 ('ROLL CALL', 46),
 ('AMOUNT', 45),
 ('VENDOR', 44),
 ('DESCRIPTION', 44),
 ('REAL ESTATE', 40),
 ('PRIVATE EQUITY', 40),
 ('INFRASTRUCTURE', 40),
 ('ATTENDEES', 37),
 ('DFA ISCV', 27),
 ('WAMCO FICP', 26),
 ('AGENDA ITEMS', 26),
 ('CTPF (TO OFFSET OVRPYMT)', 25),
 ("PUBLIC SCHOOL TEACHERS' PENSION AND RETIREMENT FUND OF CHICAGO", 24),
 ('AGENDA', 24),
 ('AT&T', 22),
 ('MSCI EAFE', 20),
 ('MANAGER OF MANAGER COMPOSITE', 20),
 ('SUB TEACHER POSTAL', 18),
 ('FY 2019', 16),
 ('FY 2017', 16),
 ('FY 2018', 16),
 ('M9', 15),
 ('DISCUSSIONS', 15),
 ('M7', 13),
 ('CURIE METRO H S', 13),
 ('MAY 16', 13),
 ('PHYSICIANS, DISABILITY APPLICANTS', 12),
 ('DISCUSSIONS/ACTIONS', 12),
 ('REPORT OF THE COMMITTEE ON CLAIMS AND SERVICE CREDITS', 11),
 ('PUBLIC PARTICIPATION', 10),
